In [ ]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import copy
from sklearn.cluster import KMeans
from joblib import Parallel, delayed, parallel_backend
import multiprocessing
import gc

# Utility Functions
def min_max_scale(data):
    data = np.array(data, dtype=float) 
    data_min = np.min(data, axis=0)
    data_max = np.max(data, axis=0)
    return (data - data_min) / (data_max - data_min + 1e-9)

def elbow_kmeans(data, max_k=5):
    inertias = []
    k_range = range(2, max_k + 1)
    for k in k_range:
        km = KMeans(n_clusters=k, n_init=2, random_state=42)
        km.fit(data)
        inertias.append(km.inertia_)
    deltas = np.diff(inertias)
    second_deriv = np.diff(deltas)
    best_k = np.argmax(second_deriv) + 2
    km_final = KMeans(n_clusters=best_k, n_init=3, random_state=42)
    labels = km_final.fit_predict(data)
    clusters = [[] for _ in range(best_k)]
    for i, label in enumerate(labels):
        clusters[label].append(i)
    return clusters

def Cluster_Retriever(original_matrix, group_a_idx, group_b_idx, max_k=10):
    group_a_data = original_matrix[:, group_a_idx]
    group_b_data = original_matrix[:, group_b_idx]
    cluster_a = elbow_kmeans(group_a_data, max_k)
    cluster_b = elbow_kmeans(group_b_data, max_k)
    return cluster_a, cluster_b

def MI_Calc(partition_P, partition_Q, n_samples):
    H_P = 0.0
    for cluster in partition_P:
        p_i = len(cluster) / n_samples
        if p_i > 0:
            H_P += -p_i * np.log(p_i)
    H_P_given_Q = 0.0
    for q_cluster in partition_Q:
        q_size = len(q_cluster)
        p_q = q_size / n_samples
        q_set = set(q_cluster)
        for p_cluster in partition_P:
            intersect_size = len(q_set.intersection(p_cluster))
            if intersect_size > 0:
                p_i_given_q = intersect_size / q_size
                H_P_given_Q += -p_q * p_i_given_q * np.log(p_i_given_q)
    return H_P - H_P_given_Q

def Mutual_Information(Complete_Dataset, group_a_idx, group_b_idx, max_k_to_be_checked=10):
    cluster_a, cluster_b = Cluster_Retriever(Complete_Dataset, group_a_idx, group_b_idx, max_k_to_be_checked)
    score = MI_Calc(cluster_a, cluster_b, n_samples=Complete_Dataset.shape[0])
    return score

def compute_mi_joblib(partition):
    group1 = [x - 1 for x in partition[1]]
    group2 = [x - 1 for x in partition[2]]
    cost = Mutual_Information(Sensor_Dataset, group1, group2, 4)
    gc.collect()
    return partition, cost

# def create_ant_and_compute_mi(pheromone_matrix, index_matrix, sensor_data):
#     print("New Ant",'\n')
#     partition = {1: [], 2: []}
#     while True:
#         partitioner(pheromone_matrix, index_matrix)
#         if len(partition[1]) + len(partition[2]) == Random_Matrix.shape[0]:
#             break
#     cost = Mutual_Information(sensor_data, [x - 1 for x in partition[1]], [x - 1 for x in partition[2]], 6)
#     gc.collect()
#     return partition, cost

def partitioner(Matrix, Index_Matrix):
    global partition
    n = Matrix.shape[0]
    idx = 1
    index = []
    for i in range(n):
        for j in range(n):
            index.append(idx)
            idx += 1
    probabilities = []
    for i in range(n):
        for j in range(n):
            probabilities.append(Matrix[i][j] / Matrix.sum())
            idx += 1
    Index_Trail = np.random.choice(index, 1, p=probabilities)[0]
    c = Index_Trail % n
    if c == 0:
        r, c = (Index_Trail // n, n)
    else:
        r, c = ((Index_Trail // n) + 1, Index_Trail % n)
    sensor1, sensor2 = Nom_dict[Index_Matrix[r - 1][c - 1]]
    if ((sensor1 not in partition[2]) and (sensor2 not in partition[1]) and
        ((sensor1 not in partition[1]) or (sensor2 not in partition[2]))):
        if sensor1 in partition[1]:
            partition[2].append(sensor2)
        elif sensor2 in partition[2]:
            partition[1].append(sensor1)
        else:
            partition[1].append(sensor1)
            partition[2].append(sensor2)

def main():
    global Pheromone_Matrix, ants, partition

    # Load and prepare data
    df1 = pd.read_csv(r"C:\Users\Administrator\Downloads\2016WT6_ALL.csv")
    df2 = pd.read_csv(r"C:\Users\Administrator\Downloads\2017WT6_ALL.csv")

    grd_power_cols = [col for col in df1.columns if col.startswith('Grd_') and 'Pwr' in col and col != 'Grd_Prod_Pwr_Avg']
    min_max_std_cols = [col for col in df1.columns if any(stat in col for stat in ['_Min', '_Max', '_Std'])]
    unwanted_wind_dir_cols = ['Amb_WindDir_Abs_Avg', 'Nac_Direction_Avg']
    drop_cols = list(set(grd_power_cols + min_max_std_cols + unwanted_wind_dir_cols + ['Prod_LatestAvg_ActPwrGen2','Turbine_ID','Timestamp','Grd_Prod_PsbleInd_Avg', 'Grd_Prod_PsbleCap_Avg', 'Prod_LatestAvg_TotActPwr', 'Prod_LatestAvg_ReactPwrGen0', 'Prod_LatestAvg_ReactPwrGen1', 'Prod_LatestAvg_ReactPwrGen2', 'Prod_LatestAvg_TotReactPwr', 'Grd_Prod_Pwr_Avg', 'Grd_Prod_PsbleInd_Avg', 'Grd_Prod_PsbleCap_Avg', 'Amb_WindSpeed_Est_Avg', 'Prod_LatestAvg_TotActPwr']))

    df1_reduced = df1.drop(columns=drop_cols)
    df2_reduced = df2.drop(columns=drop_cols)
    df2_reduced['Grd_Prod_CosPhi_Avg'] = df2_reduced['Grd_Prod_CosPhi_Avg'].fillna(df2_reduced['Grd_Prod_CosPhi_Avg'].mean())
    df_reduced = pd.concat([df1_reduced, df2_reduced], axis=0)

    global Sensor_Dataset
    Sensor_Dataset = min_max_scale(df_reduced)

    # Initialization
    global Pheromone_Matrix, Random_Matrix, Index_Matrix, Nom_dict
    Pheromone_Matrix = np.zeros((40, 40))
    Random_Matrix = np.ones((40, 40))
    np.fill_diagonal(Random_Matrix, 0)
    Index_Matrix = np.zeros((40, 40))
    Nom_dict = {}
    idx = 1
    for i in range(40):
        for j in range(40):
            Nom_dict[idx] = (i+1, j+1)
            Index_Matrix[i][j] = idx
            idx += 1

    global ants
    ants = []
    global partition
    partition = {1: [], 2: []}
    n_ants = 200
    n_jobs = min(multiprocessing.cpu_count(), 16)

    with parallel_backend("threading"):
        while len(ants) < n_ants:
            partition = {1: [], 2: []}
            while len(partition[1]) + len(partition[2]) < Random_Matrix.shape[0]:
                partitioner(Random_Matrix, Index_Matrix)
            p1, p2 = sorted(partition[1]), sorted(partition[2])
            if p1 > p2:
                p1, p2 = p2, p1
            norm = {1: p1, 2: p2}
            if norm not in ants:
                ants.append(copy.deepcopy(norm))
        print("Initial Solution Guesses :",'\n')
        for i in ants:
            print(i,'\n')

        mi_results = Parallel(n_jobs=n_jobs,backend='loky')(
            delayed(compute_mi_joblib)(ant) for ant in ants
        )

        for part, cost in mi_results:
            for i in part[1]:
                for j in part[2]:
                    Pheromone_Matrix[i - 1][j - 1] += cost
                    Pheromone_Matrix[j - 1][i - 1] += cost
    print("Initial Pheromone Matrix :",Pheromone_Matrix,'\n')

    # ACO Iterations
    iterations = 500
    best_cost = 0
    best_partition = {}
    max_costs = []
    history = []
    iterations_to_capture=[50,100,150,200,250,300,350,400,450,500]

    for it in range(1, iterations + 1):
        Pheromone_Matrix *= 0.65
        ants=[]
        costs=[]
        with parallel_backend("threading"):
            while len(ants) < (n_ants-30):
                partition = {1: [], 2: []}
                while len(partition[1]) + len(partition[2]) < Random_Matrix.shape[0]:
                    partitioner(Pheromone_Matrix, Index_Matrix)
                p1, p2 = sorted(partition[1]), sorted(partition[2])
                if p1 > p2:
                    p1, p2 = p2, p1
                norm = {1: p1, 2: p2}
                if norm not in ants:
                    ants.append(copy.deepcopy(norm))
                    
            while len(ants) < (n_ants):
                partition = {1: [], 2: []}
                while len(partition[1]) + len(partition[2]) < Random_Matrix.shape[0]:
                    partitioner(Pheromone_Matrix, Index_Matrix)
                p1, p2 = sorted(partition[1]), sorted(partition[2])
                if p1 > p2:
                    p1, p2 = p2, p1
                norm = {1: p1, 2: p2}
                if norm not in ants:
                    ants.append(copy.deepcopy(norm))
                    
            mi_results = Parallel(n_jobs=n_jobs,backend='loky')(
                delayed(compute_mi_joblib)(ant) for ant in ants
                )
            for part, cost in mi_results:
                costs.append(cost)
                for i in part[1]:
                    for j in part[2]:
                        Pheromone_Matrix[i - 1][j - 1] += cost
                        Pheromone_Matrix[j - 1][i - 1] += cost
                        
        max_iter_cost = max(costs)
        max_costs.append(max_iter_cost)
        if max_iter_cost > best_cost:
            best_cost = max_iter_cost
            best_partition = ants[costs.index(max_iter_cost)]
        history.append(best_cost)
        print(f"Iteration {it}: Max Cost = {max_iter_cost:.4f}, Best so far = {best_cost:.4f}")
        if it % 80 == 0:
            noise_scale = 0.05 * np.mean(Pheromone_Matrix)
            noise = np.random.normal(scale=noise_scale, size=Pheromone_Matrix.shape)
            Pheromone_Matrix += noise
         
        if it in iterations_to_capture:
            if it in iterations_to_capture:
                plt.figure(figsize=(25, 25))
                sns.heatmap(Pheromone_Matrix,
                            annot=True,
                            fmt=".2f",
                            cmap='viridis',
                            linewidths=0.5,
                            linecolor='gray',
                            cbar=True,
                            square=True,
                            annot_kws={"size": 10})
                plt.xticks(fontsize=10, rotation=90)
                plt.yticks(fontsize=10)
                plt.tight_layout()
                plt.savefig(f"pheromone_matrix_iter_{it}.png", dpi=300)
                plt.close()

        del ants
        del costs
        if it%5 ==0:
            gc.collect()
    print("Best Solution Found:", best_partition)
    print("Best Cost:", best_cost)
    plt.figure(figsize=(25,25))
    plt.plot(history)
    plt.xlabel("Iteration")
    plt.ylabel("Best Cost So Far")
    plt.title("Best Cost Function possible v/s iteration")
    plt.savefig("ACO_Convergence_1.png")
    plt.close()
    plt.figure(figsize=(25,25))
    plt.plot(max_costs)
    plt.xlabel("Iteration")
    plt.ylabel("Best Cost So Far")
    plt.title("Best Solution at each iteration")
    plt.savefig("ACO_Convergence_2.png")
    plt.close()

if __name__ == "__main__":
    main()




